In [1]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence

In [2]:
# number of distinct words
vocabulary_size = 10000

# number of words per review
max_review_length = 500

# load Keras IMDB movie reviews dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print('Number of reviews', len(X_train))
print('Length of first review before padding', len(X_train[0]))
print('First review', X_train[0])
print('First label', y_train[0])


Number of reviews 25000
Length of first review before padding 218
First review [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 

In [3]:
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print('Length of first review after padding', len(X_train[0]))

Length of first review after padding 500


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
import pandas as pd
embedding_dim = 5



In [7]:
model_cnn = Sequential()
model_cnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_cnn.add(Conv1D(filters=200, kernel_size=3, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(250, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_cnn = model_cnn.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=2, batch_size=64)
scores_cnn = model_cnn.evaluate(X_test, y_test, verbose=0)

Epoch 1/2
391/391 [==============================] - 86s 221ms/step - loss: 0.4447 - accuracy: 0.7668 - val_loss: 0.2970 - val_accuracy: 0.8752
Epoch 2/2
391/391 [==============================] - 86s 221ms/step - loss: 0.2155 - accuracy: 0.9158 - val_loss: 0.3236 - val_accuracy: 0.8660


In [10]:
df = pd.DataFrame({'Model': ['CNN'],
                   'Accuracy': [str(round(scores_cnn[1]*100, 2)) + '%']})

In [11]:
df

,Model,Accuracy
0,CNN,86.6%


In [16]:
# Save CNN Model
model_cnn.save('model_cnn.h5', include_optimizer=True)
json_string = model_cnn.to_json()
f = open("model_cnn_architecture.json",'w')
f.write(json_string)
f.close()

